In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture("volleyball_clip.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("volleyball_tracking.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

background_frames = 50
frame_samples = []

for _ in range(background_frames):
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_samples.append(frame)

background = np.median(np.array(frame_samples), axis=0).astype(np.uint8)

subtractor = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # movement_mask = np.any(np.abs(frame_rgb - background) > 20, axis=2).astype(np.uint8) * 255  # Convert to 255 for display
    
    movement_mask = subtractor.apply(frame_rgb)

    # color_mask_value = 0.5 * frame_rgb[:, :, 0] + 0.5 * frame_rgb[:, :, 1] - frame_rgb[:, :, 2]
    # color_mask = ((color_mask_value - np.min(color_mask_value)) / (np.max(color_mask_value) - np.min(color_mask_value)) > 0.77).astype(np.uint8) * 255
    
    # use HSV color space to detect yellow
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([90, 150, 90])
    upper_yellow = np.array([100, 255, 255])
    color_mask = cv2.inRange(frame_hsv, lower_yellow, upper_yellow)

    mask = (movement_mask & color_mask).astype(np.uint8) * 255

    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25)))

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        center = (int(x + w / 2), int(y + h / 2))
        cv2.circle(frame, center, 10, (0, 0, 255), -1)

    cv2.imshow("Movement Mask", movement_mask)
    cv2.imshow("Color Mask", color_mask)
    cv2.imshow("Final Mask (Movement & Color)", mask)
    cv2.imshow("Original", frame)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
